# Assignment 6 - Personality vs Shoes

The model will use a CNN to predict the personality associated to your favorite shoe. 

We will first train a CNN to recognize shoes based on a given image, then use the result to make a prediction on the personality. 

The model is trained to recognized multiple types of shoes:
1. 5 types of boots
2. 3 types of sandals
3. 10 types of shoes
4. 2 types of slippers

The model then uses the results of the image recognition to predict the personality that someone might have.

In [13]:
# Importing Libraries
import tensorflow as tf
from tensorflow import keras
from keras import preprocessing

# Data Preprocessing

In [14]:
import tensorflow as tf

data_dir = "Dataset"

# Create an ImageDataGenerator with a validation split
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,  # 20% of the data will be used for validation
    rotation_range=30,  # Rotate up to 30 degrees
    width_shift_range=0.2,  # Shift width by 20%
    height_shift_range=0.2,  # Shift height by 20%
    shear_range=0.2,  # Shear transformation
    brightness_range=(0.7, 1.3),  # Adjust brightness
    channel_shift_range=30.0,  # Adjust channel shifts
    fill_mode='nearest',  # Filling mode
    zoom_range=0.2,  # Random zoom
    horizontal_flip=True,  # Horizontal flipping
    vertical_flip=True  # Vertical flipping
)

# Fit the generator on the training data if you use normalization or centering
# datagen.fit(train_data)  # Optional

# Load training data
train_data = datagen.flow_from_directory(
    data_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training',  # Specify this as training set
    shuffle=True  # Shuffle the data to improve training
)

# Load validation data
val_data = datagen.flow_from_directory(
    data_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation',  # Specify this as validation set
    shuffle=True  # Shuffle the data to improve validation
)

# Optional: Check the number of images in each set
print(f"Number of training images: {train_data.samples}")
print(f"Number of validation images: {val_data.samples}")


Found 40011 images belonging to 4 classes.
Found 10000 images belonging to 4 classes.
Number of training images: 40011
Number of validation images: 10000


# Building the CNN Model

In [15]:
import tensorflow as tf

# Define the number of classes
num_classes = 15

# Initialize CNN Model
ShoeModel = tf.keras.models.Sequential()

# Add an Input layer
ShoeModel.add(tf.keras.layers.Input(shape=[150, 150, 3]))

# First Convolution layer
ShoeModel.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
ShoeModel.add(tf.keras.layers.BatchNormalization())  # Optional: batch normalization
ShoeModel.add(tf.keras.layers.MaxPooling2D(pool_size=2, strides=2))

# Second Convolution layer
ShoeModel.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu'))
ShoeModel.add(tf.keras.layers.BatchNormalization())  # Optional: batch normalization
ShoeModel.add(tf.keras.layers.MaxPooling2D(pool_size=2, strides=2))

# Third Convolution layer
ShoeModel.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu'))
ShoeModel.add(tf.keras.layers.BatchNormalization())  # Optional: batch normalization
ShoeModel.add(tf.keras.layers.MaxPooling2D(pool_size=2, strides=2))

# Flattening
ShoeModel.add(tf.keras.layers.Flatten())

# Fully connected Dense layer
ShoeModel.add(tf.keras.layers.Dense(units=128, activation='relu'))
ShoeModel.add(tf.keras.layers.Dropout(0.5))  # Dropout to prevent overfitting

# Output layer
ShoeModel.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))

# Compile the model
ShoeModel.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Model Checkpoint to save the best model during training
checkpoint = tf.keras.callbacks.ModelCheckpoint('best_model.keras', monitor='val_accuracy', save_best_only=True)

# Summary of the model
ShoeModel.summary()

# Define early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    patience=3, 
    restore_best_weights=True
)

# Fit the model (example usage)
ShoeModel.fit(x=train_data, validation_data=val_data, epochs=20)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_11 (Conv2D)              │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_11          │ (None, 148, 148, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_12          │ (None, 72, 72, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_13          │ (None, 34, 34, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 36992)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 128)            │     4,735,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 15)             │         1,935 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,831,183 (18.43 MB)

 Trainable params: 4,830,735 (18.43 MB)

 Non-trainable params: 448 (1.75 KB)

Epoch 1/20


ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(None, 4), output.shape=(None, 15)

# Train the model

In [ ]:
# Define early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    patience=3, 
    restore_best_weights=True
)

# Fit the model (example usage)
history = ShoeModel.fit(
    train_data,
    validation_data=val_data,
    epochs=20,
    callbacks=[early_stopping, checkpoint]  # Adding checkpoint callback
)

c:\Users\kylea\anaconda3\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


OSError: [Errno 22] Invalid argument